### Figure 8 / Task Setup
  * 1100 documents uploaded, of which 8 test questions were made. These are used  to establish workers' trustworthiness. However, it was hard to find documents that contained AMs, since they are so sparsely distributed. I didn't count how many documents I checked, but I'd say it was around 100 to find 7 documents that contained an AM.
  * 569 trusted judgments obtained in about 3 hours at $96.60, covering 7.9% of the data at 3 judments per document. (Some of the judgment    s were used on test questions, so 87 reports that were not test questions were actually annotated.) 
  * Workers rated the job 1.3/5 for ease (which is quite low)
  * For 22 documents, we received annotations (that's 14 new ones and 8 on the test questions)

In [188]:
import pandas
import numpy
import json
import requests

import jsonpath_ng as jp
from jsonpath_ng.ext import parse

interesting_columns = ['_unit_id','_golden','_trusted_judgments','annotation','annotation:confidence','text_raw']
dataf = pandas.read_csv('data/all.csv')[interesting_columns]
dataf = dataf[dataf.annotation.notnull()]

pandas.set_option('display.max_colwidth', 50)
dataf.head(3)

,_unit_id,_golden,_trusted_judgments,annotation,annotation:confidence,text_raw
1,2684649280,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,1.0000,"Hi , . I was diagnosed in 2005 , I was 44 work..."
8,2684649287,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,1.0000,1 . Screening all donated blood for Zika virus...
10,2684649289,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,0.8333,"Hey all , 31 year old male who was diagnosed o..."


In [189]:
dataf['json'] = dataf.apply(lambda row: requests.get(row['annotation']).json(), axis = 1)
dataf.to_csv('data/annotations.csv')
dataf.head(3)    

,_unit_id,_golden,_trusted_judgments,annotation,annotation:confidence,text_raw,json
1,2684649280,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,1.0000,"Hi , . I was diagnosed in 2005 , I was 44 work...","{'id': '2684649280', 'tokens': [{'text': 'Hi',..."
8,2684649287,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,1.0000,1 . Screening all donated blood for Zika virus...,"{'id': '2684649287', 'tokens': [{'text': '1', ..."
10,2684649289,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,0.8333,"Hey all , 31 year old male who was diagnosed o...","{'id': '2684649289', 'tokens': [{'text': 'Hey'..."


In [190]:
human_annotated = parse('''$.spans[?(@.annotated_by=='human')]''')
annotated_text = parse('''$..text''')

def get_token(myjson):
    myjson = json.loads(myjson.to_json())['json']
    matches = [ match.value for match in human_annotated.find(myjson) ][0]
    return ' '.join([match.value for match in annotated_text.find(matches)])


dataf['annotation:token'] = dataf[['json']].apply(get_token,axis=1)
pandas.set_option('display.max_colwidth', 50)
dataf.head(3)

,_unit_id,_golden,_trusted_judgments,annotation,annotation:confidence,text_raw,json,annotation:token
1,2684649280,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,1.0000,"Hi , . I was diagnosed in 2005 , I was 44 work...","{'id': '2684649280', 'tokens': [{'text': 'Hi',...",44
8,2684649287,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,1.0000,1 . Screening all donated blood for Zika virus...,"{'id': '2684649287', 'tokens': [{'text': '1', ...",infants
10,2684649289,False,3,https://f8-text-annotation-live.s3.amazonaws.c...,0.8333,"Hey all , 31 year old male who was diagnosed o...","{'id': '2684649289', 'tokens': [{'text': 'Hey'...",31 year old


In [192]:
pandas.set_option('display.max_colwidth', None)
display(dataf[['text_raw','annotation:token','_golden', 'annotation:confidence']])

text_raw  \
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Figure 8 / Evaluation

* That leaves 14 annotations that were not gold questions, of which: 
   * 6 are FP
   * 8 are TP, but 6 are the same as gold questions (but since the data has duplicates, they snuck through)
   * So 2 are the only real finds
   
* It is difficult to say how many are missed, because we don't know how many MAs there are in total. In any case, the annotations do not seem very reliable, we'd have to flick through them again.
   